In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import requests
from keras.models import Sequential
from keras.losses import MeanSquaredError
from keras.layers import Dense, Flatten, Dropout, rnn, LSTM, GRU
from keras import optimizers, datasets
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints import instruments

In [ ]:
# Use your brokers api key below

accountID = "101-001-2023237-023"

oanda = oandapyV20.API(environment="practice", access_token="dfa70843fb9dcf028cdcb-c32d9c47d102003af0")

client = oandapyV20.API(access_token=oanda.access_token)

# Data pairs of majors , minors, and those combined, which = 28 pairs
currency_pairs = ["EUR_USD", "USD_JPY", "GBP_USD", "USD_CHF", "USD_CAD", "AUD_USD",
                  "NZD_USD", "EUR_GBP", "EUR_JPY", "GBP_JPY", "EUR_CHF", "EUR_CAD",
                  "AUD_JPY", "AUD_NZD", "AUD_CAD", "AUD_CHF", "CAD_JPY", "CAD_CHF",
                  "CHF_JPY", "EUR_NZD", "GBP_AUD", "GBP_CAD", "GBP_CHF", "GBP_NZD",
                  "NZD_CAD", "NZD_CHF", "NZD_JPY", "EUR_AUD"]

instrument_list = ",".join(currency_pairs)

params = {
    "count": 700,
    "granularity": "D",
    "instruments": instrument_list
}



In [ ]:
batch_size = 28
timesteps = 700
input_dim = 5
num_candles = 5

# Generate random input data
x = np.random.rand(batch_size, timesteps, input_dim)
y = np.random.randint(0, 2, size=(batch_size,))

# Load your candle data here

instrument = params
request_object = instruments.InstrumentsCandles(instrument=instrument_list, params=params)
data = client.request(request_object)

# Extract the candle data from the response
candles = []
for candle in data['candles']:
    candles.append([candle['time'], candle['mid']['o'], candle['mid']['h'], candle['mid']['l'], candle['mid']['c']])

# Convert the candle data to a numpy array
candles = np.array(candles)

# Create a new axis for the candles data
candles = candles[np.newaxis, :, :]

# Repeat the candles data for each sample in the batch
candles = np.repeat(candles, batch_size, axis=0)

# Concatenate the candle data with your input data
x = np.concatenate((x, candles), axis=2)

x_train, x_test, y_train, y_test = train_split_test(x,y, test_size=0.2)




AttributeError: ignored

In [ ]:
 """ This function trades each pair on the currency list, using 2% of the balance, that way you will never have a margin call,
it sets a stoploss at 1.5 * atr on each trade each trade, traded. Once in a trade it will not repeat a pair.
Example: If i trade the EUR/USD in a long or buy position. This function will NOT trade any other pair that has the currency EUR or USD.
In other words it doesn't repeat traded currency. """

"""Add your trading strategy here, you can use it to train your model or use
it in python on your practice accounts on your trading platform. Feel free
to use it as you like.
"""
class forex_trading_strategies():
  def __init__(self, alpha=1.0):
    self.alpha=alpha

  def my_trading_reward(state, action, enter_trade, exit_trade, balance, atr, traded_pairs, current_pair):
      trade_size = 0.02 * balance / (1.5 * atr)
      stop_loss = state[enter_trade] - 1.5 * atr
      position = state[enter_trade]
      if action == 1:  # enter long position
          if any(curr in current_pair for curr in traded_pairs):
              position = 0
          else:
              position = trade_size
              traded_pairs.extend(current_pair.split('/'))
      elif action == 2:  # enter short position
          if any(curr in current_pair for curr in traded_pairs):
              position = 0
          else:
              position = -trade_size
              traded_pairs.extend(current_pair.split('/'))
      elif action == 3:  # exit position
          position = 0
      if state[exit_trade] <= stop_loss:
          position = 0
      profit = (state[exit_trade] - state[enter_trade]) * position
      reward = profit
      return reward, position, traded_pairs

    # add your trading strategies below

  def trading_strategy():
    pass

  def trading_strategy_1():
    pass


In [ ]:
# build models
class my_forex_models():
  def __init__(self, alpha=1.0):
    self.alpha=alpha

  def model_with_sequential():
    seq_model = Sequential([
        LSTM(512, input_shape=(timesteps, input_dim)),
        Dense(512, activation=tf.nn.relu),
        Dense(128, activation=tf.nn.relu),
        Dropout(0.25),
        Dense(28, activation=tf.nn.softmax)
    ])
    return seq_model

  def model_with_functional():
    pass

  # i can build my model using my trading indictors
  def model_with_multi_indicators():

    # Credit for indicator functions with machine learning goes to Bing AI by Microsoft
    def linear_regression_channel(self, data, period=14):
        x = tf.range(period)
        x_mean = tf.reduce_mean(x)
        y_mean = tf.reduce_mean(data[:, -period:], axis=1)
        b = tf.reduce_sum((x - x_mean) * (data[:, -period:] - y_mean), axis=1) / tf.reduce_sum(tf.square(x - x_mean))
        a = y_mean - b * x_mean
        upper = a + b * (period - 1) + 2 * tf.math.reduce_std(data[:, -period:], axis=1)
        lower = a + b * (period - 1) - 2 * tf.math.reduce_std(data[:, -period:], axis=1)
        return upper, lower

    def smi_ergodic_indicator(self, data, n=10, n2=4, n3=10):
        hl2 = (data[:, :, 0] + data[:, :, 1]) / 2
        cmo = 100 * (hl2 - hl2[:, n-1:n]) / (hl2 + hl2[:, n-1:n])
        cmo_ema1 = tf.keras.backend.squeeze(tf.keras.layers.EMA(alpha=1/n2)(tf.expand_dims(cmo, axis=-1)), axis=-1)
        cmo_ema2 = tf.keras.backend.squeeze(tf.keras.layers.EMA(alpha=1/n3)(tf.expand_dims(cmo_ema1, axis=-1)), axis=-1)
        return cmo_ema2

    def arnaud_legoux_ma(self, data, period=9, sigma=6, offset=0.85):
        m = offset * (period - 1)
        s = period / sigma
        w = tf.exp(-((tf.range(period) - m) ** 2) / (2 * s ** 2))
        alma = tf.reduce_sum(w * data[:, -period:], axis=1) / tf.reduce_sum(w)
        return alma

    def average_true_range(self, data, period=14):
        high = data[:, :, 0]
        low = data[:, :, 1]
        close = data[:, :, 2]
        tr = tf.maximum(high - low, tf.abs(high - close), tf.abs(low - close))
        atr = tf.reduce_mean(tr, axis=1)
        for i in range(1, tr.shape[1]):
            atr = (atr * (period - 1) + tr[:, i]) / period
        return atr

    def chalkin_money_flow(self, data, period=20):
        high = data[:, :, 0]
        low = data[:, :, 1]
        close = data[:, :, 2]
        volume = data[:, :, 3]
        mf_multiplier = ((close - low) - (high - close)) / (high - low)
        mf_volume = mf_multiplier * volume
        cmf = tf.reduce_sum(mf_volume[:, -period:], axis=1) / tf.reduce_sum(volume[:, -period:], axis=1)
        return cmf




In [ ]:
# you  can make a loss function with  trading indicators

"""Feel free to add your own loss functions below"""
class custom_losses():

    # Credit goes to GoGiddyUp on GitHub. For below code. https://github.com/GoGiddyUp
    def __init__(self, alpha=1.0):
      self.alpha = alpha # This is necessary for Keras to process this as a loss function

    def __call__(self, y_true, y_pred):
        # Compute loss here. This is a simple example using mean squared error
        mse = tf.keras.losses.mean_squared_error(y_true, y_pred)
        return mse

    def your_loss(self, y_true, y_pred):
      pass

    def your_loss_1(self, y_true, y_pred):
      pass

    def your_loss_2(self, y_true, y_pred):
      pass

    # Credit for indicator functions with machine learning goes to Bing AI by Microsoft
    def multi_indicator_loss(self, y_true, y_pred):

      # combine all indicators to compute the final loss
      indicator1 = self.linear_regression_channel(y_pred)
      indicator2 = self.smi_ergodic_indicator(y_pred)
      indicator3 = self.arnaud_legoux_ma(y_pred)
      indicator4 = self.average_true_range(y_pred)
      indicator5 = self.chalkin_money_flow(y_pred)

      combined_indicator = (indicator1 + indicator2 + indicator3 + indicator4 + indicator5) / 5

      mse = tf.keras.losses.mean_squared_error(y_true, combined_indicator)

      return mse

      def linear_regression_channel(self, data, period=14):
          x = tf.range(period)
          x_mean = tf.reduce_mean(x)
          y_mean = tf.reduce_mean(data[:, -period:], axis=1)
          b = tf.reduce_sum((x - x_mean) * (data[:, -period:] - y_mean), axis=1) / tf.reduce_sum(tf.square(x - x_mean))
          a = y_mean - b * x_mean
          upper = a + b * (period - 1) + 2 * tf.math.reduce_std(data[:, -period:], axis=1)
          lower = a + b * (period - 1) - 2 * tf.math.reduce_std(data[:, -period:], axis=1)
          return (upper + lower) / 2

      def smi_ergodic_indicator(self, data, n=10, n2=4, n3=10):
          hl2 = (data[:, :, 0] + data[:, :, 1]) / 2
          cmo = 100 * (hl2 - hl2[:, n-1:n]) / (hl2 + hl2[:, n-1:n])
          cmo_ema1 = tf.keras.backend.squeeze(tf.keras.layers.EMA(alpha=1/n2)(tf.expand_dims(cmo, axis=-1)), axis=-1)
          cmo_ema2 = tf.keras.backend.squeeze(tf.keras.layers.EMA(alpha=1/n3)(tf.expand_dims(cmo_ema1, axis=-1)), axis=-1)
          return cmo_ema2

      def arnaud_legoux_ma(self, data, period=9, sigma=6, offset=0.85):
          m = offset * (period - 1)
          s = period / sigma
          w = tf.exp(-((tf.range(period) - m) ** 2) / (2 * s ** 2))
          alma = tf.reduce_sum(w * data[:, -period:], axis=1) / tf.reduce_sum(w)
          return alma

      def average_true_range(self, data, period=14):
          high = data[:, :, 0]
          low = data[:, :, 1]
          close = data[:, :, 2]
          tr = tf.maximum(high - low, tf.abs(high - close), tf.abs(low - close))
          atr = tf.reduce_mean(tr, axis=1)
          for i in range(1, tr.shape[1]):
              atr = (atr * (period - 1) + tr[:, i]) / period
          return atr

      def chalkin_money_flow(self, data, period=17):
          high = data[:, :, 0]
          low = data[:, :, 1]
          close = data[:, :, 2]
          volume = data[:, :, 3]
          mf_multiplier = ((close - low) - (high - close)) / (high - low)
          mf_volume = mf_multiplier * volume
          cmf = tf.reduce_sum(mf_volume[:, -period:], axis=1) / tf.reduce_sum(volume[:, -period:], axis=1)







In [ ]:
losses = custom_losses()

model = my_forex_models()

seq_model = model.model_with_sequential()

TypeError: ignored

In [ ]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.0001),
    loss=losses.multi_indicator_loss,
    metrics=['accuracy']
)